### 这个代码用来读取所有视频基于hist5000

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('./Data/50000/hist50000.csv', names=['userid','videoid','time','label'])

In [ ]:
video = data.videoid.unique().tolist()

In [ ]:
with open('./Data/50000/vocab.txt', 'w',encoding='UTF-8') as f:
    for i in video:
        f.write(i)
        f.write('\n')

### 这个代码用来读取用户的行为基于hist5000

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('./Data/50000/hist50000.csv', names=['userid','videoid','time','label'])

In [ ]:
user_behavior = {}

In [ ]:
for i in data.userid.unique().tolist():
    user_behavior[i] = data[data.userid==i].videoid.tolist()

In [ ]:
behavior = list(user_behavior.values())

单个用户看过的最大的视频数量为： 298

In [ ]:
num=0
for i in behavior:
    if len(i)>=num:
        num=len(i)
print('单个用户看过的最大的视频数量为：',num)

In [ ]:
with open('./Data/50000/user_behavior.txt', 'w',encoding='UTF-8') as f:
    f.write(str(user_behavior))

In [ ]:
with open('./Data/50000/behavior.txt', 'w',encoding='UTF-8') as f:
    f.write(str(list(user_behavior.values())))

### 这个代码用来产生的是用来产生预训练数据的数据集

In [ ]:
with open('./Data/50000/behavior.txt', 'r',encoding='UTF-8') as f:
    behavior = eval(f.read())

In [ ]:
pre_train_behavior = []
for i in range(len(behavior)):
    pre_train_behavior.append(behavior[i][:-1])

In [ ]:
with open('./Data/50000/pre_train_behavior.txt', 'w',encoding='UTF-8') as f:
    f.write(str(pre_train_behavior))

In [ ]:
behavior[0]

## 这个代码用来产生训练集和测试集

In [ ]:
import tensorflow as tf
import collections
import random
rng = random.Random(12345)
import pandas as pd
from sklearn.model_selection import train_test_split
import csv

In [ ]:
input_file = './Data/50000/behavior.txt'
vocab_file = './Data/50000/vocab.txt'

In [ ]:
with tf.gfile.GFile(input_file, 'r') as f:
    data = eval(f.read())

In [ ]:
for i in range(len(data)):
    if len(data[i]) > 128:
        data[i] = data[i][-128:]

In [ ]:
with open(vocab_file, 'r', encoding='UTF-8') as f:
    vocab_ls = f.read()
    vocab_ls = vocab_ls.strip('\n').split('\n')

In [ ]:
class FullTokenizer():
    
    def __init__(self, vocab):
        self.vocab = vocab
        self.vocab_dt = self.reflect_vocab()
        
    def reflect_vocab(self):
        self.vocab_dt = collections.OrderedDict()
        index = 0
        for i in range(len(self.vocab)):
            self.vocab_dt[self.vocab[i]] = index
            index +=1
        return self.vocab_dt
    
    def convert_tokens_to_ids(self, tokens):
        ids = []
        for token in tokens:
            ids.append(self.vocab_dt[token])
        return ids

In [ ]:
tokenizer = FullTokenizer(vocab_ls)

In [ ]:
vocab_words = list(tokenizer.vocab_dt.keys())

In [ ]:
data_all_pos = []
data_all_neg = []
data_all = []
for i in range(len(data)):
    data_pos = data[i]
    data_neg = data[i]
    while True:
        random_index = rng.randint(0, len(tokenizer.vocab_dt.keys()))
        if vocab_words[random_index] not in data_pos:
            break
    data_neg[-1] = vocab_words[random_index]
    #打包正负样本
    data_all_pos.append(['1', data_pos])
    data_all_neg.append(['0', data_neg])
    data_all.append(['1', data_pos])
    data_all.append(['0', data_neg])
rng.shuffle(data_all)
rng.shuffle(data_all_pos)
rng.shuffle(data_all_neg)

In [ ]:
data_all_pos_DF = pd.DataFrame(data_all_pos, columns=['label', 'value'])

In [ ]:
data_all_pos_DF.to_csv('./Data/50000/data_all_pos_DF.csv', sep='\t', index=False)

In [ ]:
data_all_neg_DF = pd.DataFrame(data_all_neg, columns=['label', 'value'])

In [ ]:
data_all_neg_DF.to_csv('./Data/50000/data_all_neg_DF.csv', sep='\t', index=False)

In [ ]:
data_all = pd.DataFrame(data_all, columns=['label', 'value'])

In [ ]:
data_all.to_csv('./Data/50000/data_all.csv', sep='\t', index=False)

In [ ]:
dfTrain, dfTest = train_test_split(data_all, random_state=12345, test_size=0.2)

In [ ]:
len(dfTrain)

In [ ]:
len(dfTest)

In [ ]:
dfTrain.to_csv('./Data/50000/dfTrain.csv', sep='\t', index=False)

In [ ]:
dfTest.to_csv('./Data/50000/dfTest.csv', sep='\t', index=False)

In [ ]:
dfTrain['label'][0]

In [ ]:
with tf.gfile.Open('./Data/50000/dfTrain.csv', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    lines = []
    for line in reader:
        lines.append(line)

In [ ]:
lines[1]

In [ ]:
for (i,line) in enumerate(lines):
    if i==0:
        continue
    label = eval(line[0])
    text_a = eval(line[1])[:-1]
    next_item = eval(line[1])[-1]
    print(label)

In [ ]:
len(lines)

### 把长度限制在129

In [ ]:
import pandas as pd
import numpy as np
input_file = './Data/50000/behavior.txt'

In [ ]:
with open(input_file, 'r') as f:
    data = eval(f.read())
for i in range(len(data)):
    if len(data[i])>129:
        data[i] = data[i][-129:]

In [ ]:
with open('./Data/50000/129_behavior.txt', 'w') as f:
    f.write(str(data))

### 产生next_item的训练集和测试集

In [ ]:
import pandas as pd
import numpy as np
input_file = './Data/50000/129_behavior.txt'
vocab_file = './Data/50000/vocab.txt'
import random

In [ ]:
with open(input_file, 'r') as f:
    data = eval(f.read())

In [ ]:
with open(vocab_file, 'r', encoding='UTF-8') as f:
    vocab_ls = f.read()
    vocab_ls = vocab_ls.strip('\n').split('\n')

In [ ]:
train_set = []
test_set = []
for index in range(len(data)):
    pos_list = data[index]
    def gen_neg():
        neg = pos_list[0]
        while neg in pos_list:
            neg = random.choice(vocab_ls)
        return neg
    neg_list = [gen_neg() for i in range(len(pos_list))]
    
    for i in range(1, len(pos_list)):
        hist_i = pos_list[:i]
        if i != len(pos_list) -1:
            train_set.append((hist_i, pos_list[i], 1))
            train_set.append((hist_i, neg_list[i], 0))
        else:
            label = (pos_list[i], neg_list[i])
            test_set.append((hist_i, label))
            
random.shuffle(train_set)
random.shuffle(test_set)

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
with open('./Data/50000/train_set.txt', 'w', encoding='UTF-8') as f:
    f.write(str(train_set))

In [ ]:
with open('./Data/50000/test_set.txt', 'w', encoding='UTF-8') as f:
    f.write(str(test_set))

In [ ]:
with open('./Data/50000/repeat/repeat_train_set.txt', 'w', encoding='UTF-8') as f:
    f.write(str(train_set[:1000]))

### 有需要就运行的调试代码

In [1]:
import numpy as np
import pandas as pd

In [4]:
a = []
with open('./Data/50000/train_set.txt', 'r', encoding='UTF-8') as f:
    while True:
        data = f.readline()
        if not data:
            break
        print(len(data[0]))

1


In [ ]:
a = []
for i in range(len(data)):
    a.append(len(data[i][0]))